In [1]:
import torch
torch.cuda.is_available(), torch.cuda.get_device_name(0)

(True, 'Tesla T4')

In [2]:
!pip install torch torchvision torchaudio matplotlib opencv-python tqdm

In [3]:
!pip install segmentation-models-pytorch torch torchvision albumentations opencv-python-headless matplotlib tqdm scikit-learn --quiet

In [4]:
!pip install imutils

In [5]:
!pip install torch torchvision segmentation-models-pytorch albumentations opencv-python imutils matplotlib tqdm

In [6]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import imutils

IMG_SIZE = 256

def crop_img(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    if len(cnts) == 0:
        return img
    
    c = max(cnts, key=cv2.contourArea)
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    ADD_PIXELS = 10
    new_img = img[max(extTop[1]-ADD_PIXELS, 0):extBot[1]+ADD_PIXELS,
                  max(extLeft[0]-ADD_PIXELS, 0):extRight[0]+ADD_PIXELS].copy()
    return new_img

def preprocess_dataset_and_generate_masks(source_dir, dest_dir, mask_dir):
    os.makedirs(dest_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)

    for dir_name in os.listdir(source_dir):
        src_class_dir = os.path.join(source_dir, dir_name)
        dst_class_dir = os.path.join(dest_dir, dir_name)
        msk_class_dir = os.path.join(mask_dir, dir_name)
        os.makedirs(dst_class_dir, exist_ok=True)
        os.makedirs(msk_class_dir, exist_ok=True)
        
        for img_file in tqdm(os.listdir(src_class_dir), desc=f"Processing {dir_name}"):
            img_path = os.path.join(src_class_dir, img_file)
            image = cv2.imread(img_path)
            if image is None:
                continue
            new_img = crop_img(image)
            new_img = cv2.resize(new_img, (IMG_SIZE, IMG_SIZE))
            cv2.imwrite(os.path.join(dst_class_dir, img_file), new_img)

            # Synthetic circular mask
            mask = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.uint8)
            center = (IMG_SIZE // 2, IMG_SIZE // 2)
            radius = IMG_SIZE // 4
            cv2.circle(mask, center, radius, 255, -1)
            cv2.imwrite(os.path.join(msk_class_dir, img_file), mask)

# Example usage for Kaggle
preprocess_dataset_and_generate_masks(
    source_dir="/kaggle/input/brain-tumor-mri-dataset/Training",
    dest_dir="/kaggle/working/cleaned/Training",
    mask_dir="/kaggle/working/cleaned_masks/Training"
)


Processing glioma: 100%|██████████| 1321/1321 [00:06<00:00, 195.94it/s]


In [7]:
import shutil
import random

source_dir = "/kaggle/working/cleaned/Training"
mask_source_dir = "/kaggle/working/cleaned_masks/Training"
train_split = 0.8

for cls in os.listdir(source_dir):
    files = os.listdir(os.path.join(source_dir, cls))
    random.shuffle(files)
    split_idx = int(len(files) * train_split)
    train_files = files[:split_idx]
    test_files = files[split_idx:]

    for phase, file_list in zip(["train", "test"], [train_files, test_files]):
        for file in file_list:
            os.makedirs(f"/kaggle/working/split/{phase}/images/{cls}", exist_ok=True)
            os.makedirs(f"/kaggle/working/split/{phase}/masks/{cls}", exist_ok=True)
            shutil.copy(os.path.join(source_dir, cls, file),
                        os.path.join(f"/kaggle/working/split/{phase}/images/{cls}", file))
            shutil.copy(os.path.join(mask_source_dir, cls, file),
                        os.path.join(f"/kaggle/working/split/{phase}/masks/{cls}", file))


In [8]:
train_dataset = BrainMRIDataset(
    image_dir="/kaggle/working/split/train/images",
    mask_dir="/kaggle/working/split/train/masks",
    transform=transform
)

val_dataset = BrainMRIDataset(
    image_dir="/kaggle/working/split/test/images",
    mask_dir="/kaggle/working/split/test/masks",
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)


NameError: name 'BrainMRIDataset' is not defined

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

class BrainMRIDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.images = []
        for cls in os.listdir(image_dir):
            img_path = os.path.join(image_dir, cls)
            msk_path = os.path.join(mask_dir, cls)
            for file in os.listdir(img_path):
                self.images.append((os.path.join(img_path, file), os.path.join(msk_path, file)))
        self.transform = transform

    def __len__(self): return len(self.images)

    def __getitem__(self, idx):
        img_path, mask_path = self.images[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (256, 256))
        mask = cv2.resize(mask, (256, 256))
        mask = np.expand_dims(mask, axis=-1)
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image, mask = augmented["image"], augmented["mask"]
        return image, mask.float()

# Augmentations
transform = A.Compose([
    A.Normalize(mean=(0.0,0.0,0.0), std=(1.0,1.0,1.0)),
    ToTensorV2()
])

train_dataset = BrainMRIDataset(
    "/kaggle/working/split/train/images",
    "/kaggle/working/split/train/masks",
    transform=transform
)
val_dataset = BrainMRIDataset(
    "/kaggle/working/split/test/images",
    "/kaggle/working/split/test/masks",
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


In [ ]:
import segmentation_models_pytorch as smp

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = smp.UnetPlusPlus(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation=None
).to(device)

loss_fn = smp.losses.DiceLoss(mode="binary")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
def evaluate_model(model, dataloader):
    model.eval()
    total_intersection = 0.0
    total_union = 0.0

    with torch.no_grad():
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)

            preds = model(images)
            preds = torch.sigmoid(preds)
            preds = (preds > 0.5).float()

            total_intersection += (preds * masks).sum().item()
            total_union += (preds + masks).sum().item()

    dice = (2 * total_intersection + 1e-7) / (total_union + 1e-7)
    print(f"📈 Validation Dice Score: {dice:.4f}")
    return dice


In [19]:
from tqdm import tqdm

def train(model, loader, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        loop = tqdm(loader, desc=f"Epoch {epoch+1}/{epochs}")
        for images, masks in loop:
            images, masks = images.to(device), masks.to(device)
            preds = model(images)
            loss = loss_fn(preds, masks)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            loop.set_postfix(loss=loss.item())
        print(f"Epoch {epoch+1} | Avg Loss: {total_loss/len(loader):.4f}")

train(model, train_loader, epochs=10)


Epoch 1/10: 100%|██████████| 714/714 [03:19<00:00,  3.58it/s, loss=0.992]


Epoch 1 | Avg Loss: 0.9919


Epoch 2/10: 100%|██████████| 714/714 [03:17<00:00,  3.61it/s, loss=0.992]


Epoch 2 | Avg Loss: 0.9918


Epoch 3/10: 100%|██████████| 714/714 [03:17<00:00,  3.62it/s, loss=0.992]


...

Epoch 8 | Avg Loss: 0.9918


Epoch 9/10: 100%|██████████| 714/714 [03:17<00:00,  3.62it/s, loss=0.992]


Epoch 9 | Avg Loss: 0.9918


Epoch 10/10: 100%|██████████| 714/714 [03:16<00:00,  3.63it/s, loss=0.992]

Epoch 10 | Avg Loss: 0.9918


In [ ]:
import matplotlib.pyplot as plt

def visualize_predictions(model, dataset, num_samples=5):
    model.eval()
    with torch.no_grad():
        for i in range(num_samples):
            image, true_mask = dataset[i]
            input_tensor = image.unsqueeze(0).to(device)
            pred_mask = model(input_tensor)
            pred_mask = torch.sigmoid(pred_mask).squeeze().cpu().numpy()
            pred_mask = (pred_mask > 0.5).astype(np.uint8)

            fig, axs = plt.subplots(1, 3, figsize=(12, 4))
            axs[0].imshow(image.permute(1, 2, 0).cpu())
            axs[0].set_title("Image")
            axs[1].imshow(true_mask.squeeze().cpu(), cmap='gray')
            axs[1].set_title("True Mask")
            axs[2].imshow(pred_mask, cmap='gray')
            axs[2].set_title("Predicted Mask")
            plt.show()

# 🔍 Visualize 5 predictions
visualize_predictions(model, val_dataset)


In [10]:
import matplotlib.pyplot as plt

def visualize_predictions(model, dataset, num_samples=5):
    model.eval()
    with torch.no_grad():
        for i in range(num_samples):
            image, true_mask = dataset[i]
            input_tensor = image.unsqueeze(0).to(device)
            pred_mask = model(input_tensor)
            pred_mask = torch.sigmoid(pred_mask).squeeze().cpu().numpy()
            pred_mask = (pred_mask > 0.5).astype(np.uint8)

            fig, axs = plt.subplots(1, 3, figsize=(12, 4))
            axs[0].imshow(image.permute(1, 2, 0).cpu())
            axs[0].set_title("Image")
            axs[1].imshow(true_mask.squeeze().cpu(), cmap='gray')
            axs[1].set_title("True Mask")
            axs[2].imshow(pred_mask, cmap='gray')
            axs[2].set_title("Predicted Mask")
            plt.show()

# Run on validation set
visualize_predictions(model, val_dataset)


NameError: name 'model' is not defined

In [29]:
import torch
import torch.nn.functional as F

# DICE score function
def dice_score(preds, targets, threshold=0.5, eps=1e-7):
    preds = torch.sigmoid(preds)
    preds = (preds > threshold).float()

    intersection = (preds * targets).sum(dim=(1, 2, 3))
    union = preds.sum(dim=(1, 2, 3)) + targets.sum(dim=(1, 2, 3))
    dice = (2. * intersection + eps) / (union + eps)
    return dice.mean().item()

# Evaluation loop
def evaluate_model(model, dataloader):
    model.eval()
    total_dice = 0.0
    count = 0

    with torch.no_grad():
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)

            preds = model(images)
            score = dice_score(preds, masks)
            total_dice += score
            count += 1

    avg_dice = total_dice / count
    print(f"📈 Average Dice Score on Validation Set: {avg_dice:.4f}")
    return avg_dice
   


In [ ]:
def dice_score(preds, targets, threshold=0.5, eps=1e-7):
    preds = torch.sigmoid(preds)
    preds = (preds > threshold).float()

    targets = targets.float()

    intersection = (preds * targets).sum()
    union = preds.sum() + targets.sum()
    dice = (2. * intersection + eps) / (union + eps)

    return dice.item()


In [40]:
def evaluate_model(model, dataloader):
    model.eval()
    total_intersection = 0.0
    total_union = 0.0

    with torch.no_grad():
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)

            preds = model(images)
            preds = torch.sigmoid(preds)
            preds = (preds > 0.5).float()

            total_intersection += (preds * masks).sum().item()
            total_union += (preds + masks).sum().item()

    dice = (2 * total_intersection + 1e-7) / (total_union + 1e-7)
    print(f"📈 Corrected Average Dice Score: {dice:.4f}")
    return dice


In [41]:
evaluate_model(model, val_loader)


📈 Corrected Average Dice Score: 0.9207


0.9206682850009292

In [ ]:
def evaluate_metrics(model, dataloader, threshold=0.5):
    model.eval()

    TP = 0
    FP = 0
    FN = 0
    total_iou = 0

    with torch.no_grad():
        for images, masks in dataloader:
            images, masks = images.to(device), masks.to(device)

            preds = model(images)
            preds = torch.sigmoid(preds)
            preds = (preds > threshold).float()

            TP += ((preds == 1) & (masks == 1)).sum().item()
            FP += ((preds == 1) & (masks == 0)).sum().item()
            FN += ((preds == 0) & (masks == 1)).sum().item()

            # IoU per batch
            intersection = (preds * masks).sum().item()
            union = ((preds + masks) >= 1).sum().item()
            if union > 0:
                total_iou += intersection / union

    precision = TP / (TP + FP + 1e-7)
    recall = TP / (TP + FN + 1e-7)
    iou = total_iou / len(dataloader)

    print(f"📐 Precision: {precision:.4f}")
    print(f"📏 Recall: {recall:.4f}")
    print(f"📦 IoU Score: {iou:.4f}")

    return precision, recall, iou
